In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import warnings
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
#import lightgbm as lgb
import gc
import sys
warnings.filterwarnings('ignore')
plt.style.use('fivethirtyeight')

In [2]:
#1- Cargo fichero bureau_balance.csv y veo cabecera 
BureauBalance = pd.read_csv('/Users/LENOVO/Downloads/bureau_balance.csv', delimiter= ',' , header=0)
#BureauBalance.head()
#BureauBalance
BureauBalanceSNA = BureauBalance.dropna()
BureauBalanceSNA

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C
...,...,...,...
27299920,5041336,-47,X
27299921,5041336,-48,X
27299922,5041336,-49,X
27299923,5041336,-50,X


In [3]:
BureauBalanceSNA.dtypes.value_counts()

int64     2
object    1
dtype: int64

In [5]:
#Función cuenta variables categoricas
def count_categoricas(df, group_var, df_name):
    """Calcula conteos y conteos normalizados para cada observación
    de `group_var` de cada categoría única en cada variable categórica
    
    Parámetros
    --------
    df : dataframe 
         El marco de datos para calcular el valor cuenta.
        
    group_var : string
        La variable por la cual agrupar el marco de datos. Para cada unico
        valor de esta variable, el marco de datos final tendrá una fila
        
    df_name : string
        Variable agregada al frente de los nombres de columna para realizar un seguimiento de las columnas

    
    Return
    --------
    categorical : dataframe
        Un marco de datos con recuentos y recuentos normalizados de cada categoría única en cada variable categórica
        con una fila por cada valor único de `group_var`
        
    """
    
    #Seleccion de las columnas categóricas
    categorica = pd.get_dummies(df.select_dtypes('object'))

    # Pone la de identificación en la columna
    categorica[group_var] = df[group_var]

    # Agrupa por el grupo var y calcula la suma y la media
    categorica = categorica.groupby(group_var).agg(['sum', 'mean'])
    
    column_names = []
    
    # Itera a través de las columnas en el nivel 0
    for var in categorica.columns.levels[0]:
        # Iterar a través de las estadísticas en el nivel 1
        for stat in ['count', 'count_norm']:
            # Genera un nuevo nombre de columna
            column_names.append('%s_%s_%s' % (df_name, var, stat))
    
    categorica.columns = column_names
    
    return categorica

In [6]:
#Función cuenta variables numéricas
def agg_numericas(df, group_var, df_name):
    """Agrega los valores numéricos en un marco de datos. Esto puede
    se utilizará para crear características para cada instancia de la variable de agrupación.
    
    Parameters
    --------
        df (dataframe): 
            el marco de datos para calcular las estadísticas
        group_var (string): 
            la variable por la cual agrupar df
        df_name (string):            
            la variable utilizada para renombrar las columnas
        
    Return
    --------
        agg (dataframe): 
            un marco de datos con las estadísticas agregadas para
            Todas las columnas numéricas. Cada instancia de la variable de agrupación tendrá
            las estadísticas (media, min, max, suma; actualmente admitidas) calculadas.
            Las columnas también se renombran para realizar un seguimiento de las características creadas.
    
    """
    # Elimina variables de identificación que no sean variables de agrupación
    for col in df:
        if col != group_var and 'SK_ID' in col:
            df = df.drop(columns = col)
            
    group_ids = df[group_var]
    numeric_df = df.select_dtypes('number')
    numeric_df[group_var] = group_ids

    # Agrupa por la variable especificada y calcula las estadísticas
    agg = numeric_df.groupby(group_var).agg(['count', 'mean', 'max', 'min', 'sum']).reset_index()

   #Crea nuevos nombres de columna
    columns = [group_var]

   #Itera a través de los nombres de las variables
    for var in agg.columns.levels[0]:
        # Salta la variable de agrupación
        if var != group_var:
            #Itera a través de los nombres de estadísticas
            for stat in agg.columns.levels[1][:-1]:
                # Hace un nuevo nombre de columna para la variable y estadística
                columns.append('%s_%s_%s' % (df_name, var, stat))

    agg.columns = columns
    return agg

In [7]:
#Recuento de cada tipo de estado de las variables CATEGORICAS para cada préstamo anterior por campo 'SK_ID_BUREAU'
BureauBalance_counts = count_categoricas(BureauBalanceSNA, group_var = 'SK_ID_BUREAU', df_name = 'bureau_balance')
#BureauBalance_counts.head()
BureauBalance_counts

,bureau_balance_STATUS_0_count,bureau_balance_STATUS_0_count_norm,bureau_balance_STATUS_1_count,bureau_balance_STATUS_1_count_norm,bureau_balance_STATUS_2_count,bureau_balance_STATUS_2_count_norm,bureau_balance_STATUS_3_count,bureau_balance_STATUS_3_count_norm,bureau_balance_STATUS_4_count,bureau_balance_STATUS_4_count_norm,bureau_balance_STATUS_5_count,bureau_balance_STATUS_5_count_norm,bureau_balance_STATUS_C_count,bureau_balance_STATUS_C_count_norm,bureau_balance_STATUS_X_count,bureau_balance_STATUS_X_count_norm
SK_ID_BUREAU,,,,,,,,,,,,,,,,
5001709,0,0.000000,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,86,0.886598,11,0.113402
5001710,5,0.060241,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,48,0.578313,30,0.361446
5001711,3,0.750000,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.000000,1,0.250000
5001712,10,0.526316,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,9,0.473684,0,0.000000
5001713,0,0.000000,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.000000,22,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6842884,9,0.187500,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,20,0.416667,19,0.395833
6842885,12,0.500000,0,0.000000,0,0.0,0,0.0,0,0.0,12,0.5,0,0.000000,0,0.000000
6842886,8,0.242424,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,25,0.757576,0,0.000000


In [8]:
# Calculo estadísticas de recuento de variables NUMERICAS para cada 'SK_ID_BUREAU' 
BureauBalance_agg = agg_numericas(BureauBalanceSNA, group_var = 'SK_ID_BUREAU', df_name = 'bureau_balance')
#BureauBalance_agg.head()
BureauBalance_agg

,SK_ID_BUREAU,bureau_balance_MONTHS_BALANCE_count,bureau_balance_MONTHS_BALANCE_mean,bureau_balance_MONTHS_BALANCE_max,bureau_balance_MONTHS_BALANCE_min,bureau_balance_MONTHS_BALANCE_sum
0,5001709,97,-48.0,0,-96,-4656
1,5001710,83,-41.0,0,-82,-3403
2,5001711,4,-1.5,0,-3,-6
3,5001712,19,-9.0,0,-18,-171
4,5001713,22,-10.5,0,-21,-231
...,...,...,...,...,...,...
817390,6842884,48,-23.5,0,-47,-1128
817391,6842885,24,-11.5,0,-23,-276
817392,6842886,33,-16.0,0,-32,-528
817393,6842887,37,-18.0,0,-36,-666


In [9]:
# Dataframe agrupando BureauBalance_agg con BureauBalance_counts por préstamo anterior
BureauBalance_by_loan = BureauBalance_agg.merge(BureauBalance_counts, right_index = True, left_on = 'SK_ID_BUREAU', how = 'outer')
BureauBalance_by_loan

,SK_ID_BUREAU,bureau_balance_MONTHS_BALANCE_count,bureau_balance_MONTHS_BALANCE_mean,bureau_balance_MONTHS_BALANCE_max,bureau_balance_MONTHS_BALANCE_min,bureau_balance_MONTHS_BALANCE_sum,bureau_balance_STATUS_0_count,bureau_balance_STATUS_0_count_norm,bureau_balance_STATUS_1_count,bureau_balance_STATUS_1_count_norm,...,bureau_balance_STATUS_3_count,bureau_balance_STATUS_3_count_norm,bureau_balance_STATUS_4_count,bureau_balance_STATUS_4_count_norm,bureau_balance_STATUS_5_count,bureau_balance_STATUS_5_count_norm,bureau_balance_STATUS_C_count,bureau_balance_STATUS_C_count_norm,bureau_balance_STATUS_X_count,bureau_balance_STATUS_X_count_norm
0,5001709,97,-48.0,0,-96,-4656,0,0.000000,0,0.000000,...,0,0.0,0,0.0,0,0.0,86,0.886598,11,0.113402
1,5001710,83,-41.0,0,-82,-3403,5,0.060241,0,0.000000,...,0,0.0,0,0.0,0,0.0,48,0.578313,30,0.361446
2,5001711,4,-1.5,0,-3,-6,3,0.750000,0,0.000000,...,0,0.0,0,0.0,0,0.0,0,0.000000,1,0.250000
3,5001712,19,-9.0,0,-18,-171,10,0.526316,0,0.000000,...,0,0.0,0,0.0,0,0.0,9,0.473684,0,0.000000
4,5001713,22,-10.5,0,-21,-231,0,0.000000,0,0.000000,...,0,0.0,0,0.0,0,0.0,0,0.000000,22,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
817390,6842884,48,-23.5,0,-47,-1128,9,0.187500,0,0.000000,...,0,0.0,0,0.0,0,0.0,20,0.416667,19,0.395833
817391,6842885,24,-11.5,0,-23,-276,12,0.500000,0,0.000000,...,0,0.0,0,0.0,12,0.5,0,0.000000,0,0.000000
817392,6842886,33,-16.0,0,-32,-528,8,0.242424,0,0.000000,...,0,0.0,0,0.0,0,0.0,25,0.757576,0,0.000000
817393,6842887,37,-18.0,0,-36,-666,6,0.162162,0,0.000000,...,0,0.0,0,0.0,0,0.0,31,0.837838,0,0.000000


In [10]:
#2- Cargo fichero bureau.csv
Bureau = pd.read_csv('/Users/LENOVO/Downloads/bureau.csv', delimiter= ',' , header=0)
#Bureau.head()
#Bureau
BureauSNA = Bureau.dropna()
BureauSNA

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
822,298038,5715510,Closed,currency 1,-586,0,-339.0,-553.0,0.000,0,40791.330,0.0,0.0,0.0,Consumer credit,-553,67995.000
846,200527,5715537,Closed,currency 1,-824,0,-521.0,-581.0,0.000,0,89415.000,0.0,0.0,0.0,Consumer credit,-580,8245.395
849,393321,5715540,Closed,currency 1,-146,0,935.0,-41.0,0.000,0,445262.940,0.0,0.0,0.0,Consumer credit,-35,12162.150
896,296586,5715600,Closed,currency 1,-359,0,-55.0,-55.0,0.000,0,77239.035,0.0,0.0,0.0,Consumer credit,-50,0.000
902,190334,5715607,Closed,currency 1,-2320,0,-1953.0,-1953.0,0.000,0,64851.750,0.0,0.0,0.0,Consumer credit,-1155,16200.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716121,366608,5047705,Closed,currency 1,-2504,0,-2199.0,-2199.0,0.000,0,87862.500,0.0,0.0,0.0,Consumer credit,-2199,30600.000
1716126,217655,5047824,Closed,currency 1,-1050,0,-684.0,-684.0,0.000,0,55521.000,0.0,0.0,0.0,Consumer credit,-682,0.000
1716275,129052,5053577,Closed,currency 1,-2870,0,-2564.0,-2564.0,1611.045,0,15705.000,0.0,0.0,0.0,Consumer credit,-2557,43735.500
1716285,435426,5053678,Closed,currency 1,-1147,0,-416.0,-849.0,0.000,0,205488.000,0.0,0.0,0.0,Consumer credit,-835,94378.500


In [11]:
# Fusion para incluir la SK_ID_CURR
BureauBureauBalance_by_loan = BureauBalance_by_loan.merge(BureauSNA[['SK_ID_BUREAU', 'SK_ID_CURR']], on = 'SK_ID_BUREAU', how = 'left')
#BureauBureauBalance_by_loan.head()
BureauBureauBalance_by_loan


,SK_ID_BUREAU,bureau_balance_MONTHS_BALANCE_count,bureau_balance_MONTHS_BALANCE_mean,bureau_balance_MONTHS_BALANCE_max,bureau_balance_MONTHS_BALANCE_min,bureau_balance_MONTHS_BALANCE_sum,bureau_balance_STATUS_0_count,bureau_balance_STATUS_0_count_norm,bureau_balance_STATUS_1_count,bureau_balance_STATUS_1_count_norm,...,bureau_balance_STATUS_3_count_norm,bureau_balance_STATUS_4_count,bureau_balance_STATUS_4_count_norm,bureau_balance_STATUS_5_count,bureau_balance_STATUS_5_count_norm,bureau_balance_STATUS_C_count,bureau_balance_STATUS_C_count_norm,bureau_balance_STATUS_X_count,bureau_balance_STATUS_X_count_norm,SK_ID_CURR
0,5001709,97,-48.0,0,-96,-4656,0,0.000000,0,0.000000,...,0.0,0,0.0,0,0.0,86,0.886598,11,0.113402,NaN
1,5001710,83,-41.0,0,-82,-3403,5,0.060241,0,0.000000,...,0.0,0,0.0,0,0.0,48,0.578313,30,0.361446,NaN
2,5001711,4,-1.5,0,-3,-6,3,0.750000,0,0.000000,...,0.0,0,0.0,0,0.0,0,0.000000,1,0.250000,NaN
3,5001712,19,-9.0,0,-18,-171,10,0.526316,0,0.000000,...,0.0,0,0.0,0,0.0,9,0.473684,0,0.000000,NaN
4,5001713,22,-10.5,0,-21,-231,0,0.000000,0,0.000000,...,0.0,0,0.0,0,0.0,0,0.000000,22,1.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
817390,6842884,48,-23.5,0,-47,-1128,9,0.187500,0,0.000000,...,0.0,0,0.0,0,0.0,20,0.416667,19,0.395833,NaN
817391,6842885,24,-11.5,0,-23,-276,12,0.500000,0,0.000000,...,0.0,0,0.0,12,0.5,0,0.000000,0,0.000000,NaN
817392,6842886,33,-16.0,0,-32,-528,8,0.242424,0,0.000000,...,0.0,0,0.0,0,0.0,25,0.757576,0,0.000000,NaN
817393,6842887,37,-18.0,0,-36,-666,6,0.162162,0,0.000000,...,0.0,0,0.0,0,0.0,31,0.837838,0,0.000000,NaN


In [12]:
##***** Agrega las estadísticas de prestamos por 'SK_ID_CURR'. Elimina columna 'SK_ID_BUREAU'****
BureauBureauBalance_by_client = agg_numericas(BureauBureauBalance_by_loan.drop(columns = ['SK_ID_BUREAU']), group_var = 'SK_ID_CURR', df_name = 'client')
#Bureau_balance_by_client.head()
BureauBureauBalance_by_client

,SK_ID_CURR,client_bureau_balance_MONTHS_BALANCE_count_count,client_bureau_balance_MONTHS_BALANCE_count_mean,client_bureau_balance_MONTHS_BALANCE_count_max,client_bureau_balance_MONTHS_BALANCE_count_min,client_bureau_balance_MONTHS_BALANCE_count_sum,client_bureau_balance_MONTHS_BALANCE_mean_count,client_bureau_balance_MONTHS_BALANCE_mean_mean,client_bureau_balance_MONTHS_BALANCE_mean_max,client_bureau_balance_MONTHS_BALANCE_mean_min,...,client_bureau_balance_STATUS_X_count_count,client_bureau_balance_STATUS_X_count_mean,client_bureau_balance_STATUS_X_count_max,client_bureau_balance_STATUS_X_count_min,client_bureau_balance_STATUS_X_count_sum,client_bureau_balance_STATUS_X_count_norm_count,client_bureau_balance_STATUS_X_count_norm_mean,client_bureau_balance_STATUS_X_count_norm_max,client_bureau_balance_STATUS_X_count_norm_min,client_bureau_balance_STATUS_X_count_norm_sum
0,100002.0,2,10.000000,16,4,20,2,-29.500000,-19.5,-39.5,...,2,1.500000,3,0,3,2,0.093750,0.187500,0.000000,0.187500
1,100032.0,1,19.000000,19,19,19,1,-36.000000,-36.0,-36.0,...,1,2.000000,2,2,2,1,0.105263,0.105263,0.105263,0.105263
2,100037.0,2,66.000000,66,66,132,2,-51.500000,-45.5,-57.5,...,2,0.000000,0,0,0,2,0.000000,0.000000,0.000000,0.000000
3,100043.0,1,2.000000,2,2,2,1,-85.500000,-85.5,-85.5,...,1,1.000000,1,1,1,1,0.500000,0.500000,0.500000,0.500000
4,100059.0,2,11.500000,16,7,23,2,-19.750000,-3.0,-36.5,...,2,0.000000,0,0,0,2,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24982,456219.0,1,8.000000,8,8,8,1,-3.500000,-3.5,-3.5,...,1,0.000000,0,0,0,1,0.000000,0.000000,0.000000,0.000000
24983,456234.0,2,15.500000,16,15,31,2,-17.750000,-7.0,-28.5,...,2,1.000000,1,1,2,2,0.064583,0.066667,0.062500,0.129167
24984,456238.0,1,88.000000,88,88,88,1,-43.500000,-43.5,-43.5,...,1,0.000000,0,0,0,1,0.000000,0.000000,0.000000,0.000000
24985,456244.0,3,40.333333,57,14,121,3,-19.666667,-6.5,-28.0,...,3,1.666667,2,1,5,3,0.066800,0.142857,0.017544,0.200401


In [13]:
BureauBureauBalance_by_client.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

Series([], dtype: float64)

In [14]:
Bureau.dtypes.value_counts()

float64    8
int64      6
object     3
dtype: int64

In [15]:
#2- Cargo fichero bureau.csv
#Bureau = pd.read_csv('/Users/LENOVO/Downloads/bureau.csv', delimiter= ',' , header=0)
#BureauSNA = Bureau.dropna()
BureauSNA

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
822,298038,5715510,Closed,currency 1,-586,0,-339.0,-553.0,0.000,0,40791.330,0.0,0.0,0.0,Consumer credit,-553,67995.000
846,200527,5715537,Closed,currency 1,-824,0,-521.0,-581.0,0.000,0,89415.000,0.0,0.0,0.0,Consumer credit,-580,8245.395
849,393321,5715540,Closed,currency 1,-146,0,935.0,-41.0,0.000,0,445262.940,0.0,0.0,0.0,Consumer credit,-35,12162.150
896,296586,5715600,Closed,currency 1,-359,0,-55.0,-55.0,0.000,0,77239.035,0.0,0.0,0.0,Consumer credit,-50,0.000
902,190334,5715607,Closed,currency 1,-2320,0,-1953.0,-1953.0,0.000,0,64851.750,0.0,0.0,0.0,Consumer credit,-1155,16200.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716121,366608,5047705,Closed,currency 1,-2504,0,-2199.0,-2199.0,0.000,0,87862.500,0.0,0.0,0.0,Consumer credit,-2199,30600.000
1716126,217655,5047824,Closed,currency 1,-1050,0,-684.0,-684.0,0.000,0,55521.000,0.0,0.0,0.0,Consumer credit,-682,0.000
1716275,129052,5053577,Closed,currency 1,-2870,0,-2564.0,-2564.0,1611.045,0,15705.000,0.0,0.0,0.0,Consumer credit,-2557,43735.500
1716285,435426,5053678,Closed,currency 1,-1147,0,-416.0,-849.0,0.000,0,205488.000,0.0,0.0,0.0,Consumer credit,-835,94378.500


In [16]:
#Se cuentan los prestamos anteriores por campo SK_ID_BUREAU y se agrupan por 'SK_ID_CURR'
Previous_loan_counts = BureauSNA.groupby('SK_ID_CURR', as_index=False)['SK_ID_BUREAU'].count().rename(columns = {'SK_ID_BUREAU': 'previous_loan_counts'})
Previous_loan_counts

,SK_ID_CURR,previous_loan_counts
0,100002,2
1,100032,1
2,100037,2
3,100043,1
4,100059,2
...,...,...
24985,456219,1
24986,456234,2
24987,456238,1
24988,456244,3


In [17]:
#Genera recuento y estadisticas de las variables para cada préstamo anterior eliminando columna 'SK_ID_BUREAU'y
#agrupando por campo 'SK_ID_CURR'
Bureau_agg = BureauSNA.drop(columns = ['SK_ID_BUREAU']).groupby('SK_ID_CURR',
as_index = False).agg(['count', 'mean', 'max', 'min', 'sum']).reset_index() 
#Bureau_agg.head()
Bureau_agg

SK_ID_CURR DAYS_CREDIT                                 \
                       count         mean   max   min   sum   
0         100002           2 -1041.000000  -645 -1437 -2082   
1         100032           1 -1388.000000 -1388 -1388 -1388   
2         100037           2 -2570.000000 -2379 -2761 -5140   
3         100043           1 -2639.000000 -2639 -2639 -2639   
4         100059           2  -768.500000  -194 -1343 -1537   
...          ...         ...          ...   ...   ...   ...   
24985     456219           1  -227.000000  -227  -227  -227   
24986     456234           2  -778.000000  -438 -1118 -1556   
24987     456238           1 -2665.000000 -2665 -2665 -2665   
24988     456244           3 -1205.333333  -400 -1719 -3616   
24989     456247           1  -776.000000  -776  -776  -776   

      CREDIT_DAY_OVERDUE               ... DAYS_CREDIT_UPDATE               \
                   count mean max min  ...              count         mean   
0                      2    0   0   0  ...                  2  -609.500000   
1                      1    0   0   0  ...                  1  -456.000000   
2                      2    0   0   0  ...                  2 -2439.500000   
3                      1    0   0   0  ...                  1 -2513.000000   
4                      2    0   0   0  ...                  2  -371.000000   
...                  ...  ...  ..  ..  ...                ...          ...   
24985                  1    0   0   0  ...                  1   -10.000000   
24986                  2    0   0   0  ...                  2  -681.000000   
24987                  1    0   0   0  ...                  1 -2602.000000   
24988                  3    0   0   0  ...                  3 -1056.666667   
24989                  1    0   0   0  ...                  1  -500.000000   

                        AMT_ANNUITY                                             
        max   min   sum       count       mean        max        min       sum  
0       -34 -1185 -1219           2      0.000      0.000      0.000      0.00  
1      -456  -456  -456           1  17437.500  17437.500  17437.500  17437.50  
2     -2243 -2636 -4879           2      0.000      0.000      0.000      0.00  
3     -2513 -2513 -2513           1  16290.000  16290.000  16290.000  16290.00  
4      -100  -642  -742           2  24993.000  49986.000      0.000  49986.00  
...     ...   ...   ...         ...        ...        ...        ...       ...  
24985   -10   -10   -10           1  26775.000  26775.000  26775.000  26775.00  
24986  -307 -1055 -1362           2  13125.015  13125.015  13125.015  26250.03  
24987 -2602 -2602 -2602           1  19800.000  19800.000  19800.000  19800.00  
24988  -399 -1416 -3170           3      0.000      0.000      0.000      0.00  
24989  -500  -500  -500           1   4329.000   4329.000   4329.000   4329.00  

[24990 rows x 61 columns]

In [18]:
#Funcion para cambiar nombres en el df
# Lista de nombres de columna
columns = ['SK_ID_CURR']

#Itera a través de los nombres de las variables
for var in Bureau_agg.columns.levels[0]:
    # Salta el nombre de identificación
    if var != 'SK_ID_CURR':
        
        # Iterate through the stat names
        for stat in Bureau_agg.columns.levels[1][:-1]:
            # Itera a través de los nombres de estadísticas
            columns.append('bureau_%s_%s' % (var, stat))

In [77]:
#**** Asigna la lista de nombres de columnas como los nombres de columna del marco de datos***
Bureau_agg.columns = columns
#dfBureau_agg.head()
Bureau_agg

,SK_ID_CURR,bureau_DAYS_CREDIT_count,bureau_DAYS_CREDIT_mean,bureau_DAYS_CREDIT_max,bureau_DAYS_CREDIT_min,bureau_DAYS_CREDIT_sum,bureau_CREDIT_DAY_OVERDUE_count,bureau_CREDIT_DAY_OVERDUE_mean,bureau_CREDIT_DAY_OVERDUE_max,bureau_CREDIT_DAY_OVERDUE_min,...,bureau_DAYS_CREDIT_UPDATE_count,bureau_DAYS_CREDIT_UPDATE_mean,bureau_DAYS_CREDIT_UPDATE_max,bureau_DAYS_CREDIT_UPDATE_min,bureau_DAYS_CREDIT_UPDATE_sum,bureau_AMT_ANNUITY_count,bureau_AMT_ANNUITY_mean,bureau_AMT_ANNUITY_max,bureau_AMT_ANNUITY_min,bureau_AMT_ANNUITY_sum
0,100002,2,-1041.000000,-645,-1437,-2082,2,0,0,0,...,2,-609.500000,-34,-1185,-1219,2,0.000,0.000,0.000,0.00
1,100032,1,-1388.000000,-1388,-1388,-1388,1,0,0,0,...,1,-456.000000,-456,-456,-456,1,17437.500,17437.500,17437.500,17437.50
2,100037,2,-2570.000000,-2379,-2761,-5140,2,0,0,0,...,2,-2439.500000,-2243,-2636,-4879,2,0.000,0.000,0.000,0.00
3,100043,1,-2639.000000,-2639,-2639,-2639,1,0,0,0,...,1,-2513.000000,-2513,-2513,-2513,1,16290.000,16290.000,16290.000,16290.00
4,100059,2,-768.500000,-194,-1343,-1537,2,0,0,0,...,2,-371.000000,-100,-642,-742,2,24993.000,49986.000,0.000,49986.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24985,456219,1,-227.000000,-227,-227,-227,1,0,0,0,...,1,-10.000000,-10,-10,-10,1,26775.000,26775.000,26775.000,26775.00
24986,456234,2,-778.000000,-438,-1118,-1556,2,0,0,0,...,2,-681.000000,-307,-1055,-1362,2,13125.015,13125.015,13125.015,26250.03
24987,456238,1,-2665.000000,-2665,-2665,-2665,1,0,0,0,...,1,-2602.000000,-2602,-2602,-2602,1,19800.000,19800.000,19800.000,19800.00
24988,456244,3,-1205.333333,-400,-1719,-3616,3,0,0,0,...,3,-1056.666667,-399,-1416,-3170,3,0.000,0.000,0.000,0.00


In [19]:
def agg_numeric(df, parent_var, df_name):
    """
    Agrupa y agrega los valores numéricos en un marco de datos hijo
    por la variable padre.
    
    Parameters
    --------
        df (dataframe): 
            el marco de datos hijo para calcular las estadísticas en
        parent_var (string): 
            La variable principal utilizada para agrupar y agregar
        df_name (string): 
            la variable utilizada para renombrar las columnas
        
    Return
    --------
        agg (dataframe): 
            un marco de datos con las estadísticas agregadas por el `parent_var` para
            Todas las columnas numéricas. Cada observación de la variable principal tendrá
            una fila en el marco de datos con la variable principal como índice.
            Las columnas también se renombran usando el `df_name`. Columnas con todos los duplicados.
            Se eliminan los valores.
    
    """
      
    # Elimina variables de identificación que no sean variables de agrupación
    for col in df:
        if col != parent_var and 'SK_ID' in col:
            df = df.drop(columns = col)
            
    # Solo toma las variables numéricas
    parent_ids = df[parent_var].copy()
    numeric_df = df.select_dtypes('number').copy()
    numeric_df[parent_var] = parent_ids
    
    # Agrupa por la variable especificada y calcula las estadísticas
    agg = numeric_df.groupby(parent_var).agg(['count', 'mean', 'max', 'min', 'sum'])

    # Necesita crear nuevos nombres de columna
    columns = []

    # Itera a través de los nombres de las variables
    for var in agg.columns.levels[0]:
        if var != parent_var:
            # Itera a través de los nombres de estadísticas
            for stat in agg.columns.levels[1]:
                # Genera un nuevo nombre de columna para la variable y estadística
                columns.append('%s_%s_%s' % (df_name, var, stat))
    
    agg.columns = columns
    
    # Elimina las columnas con todos los valores redundantes
    _, idx = np.unique(agg, axis = 1, return_index=True)
    agg = agg.iloc[:, idx]
    
    return agg

In [20]:
def agg_categorical(df, parent_var, df_name):
    """
    Agrega las características categóricas en un marco de datos hijo
    para cada observación de la variable principal.
    
    Parameters
    --------
    df : dataframe 
        El marco de datos para calcular el valor cuenta.
        
    parent_var : string
        La variable por la cual agrupar y agregar el marco de datos. Para cada unico
        valor de esta variable, el marco de datos final tendrá una fila
        
    df_name : string
       Variable agregada al frente de los nombres de columna para realizar un seguimiento de las columnas

    
    Return
    --------
    categorical : dataframe
        Un marco de datos con estadísticas agregadas para cada observación de parent_var
        Las columnas también se renombran y las columnas con valores duplicados se eliminan.
        
    """
       
    # Selecciona las columnas categóricas
    categorical = pd.get_dummies(df.select_dtypes('category'))

    # Asegura poner la identificación de identificación en la columna
    categorical[parent_var] = df[parent_var]

    # Agrupa por el grupo var y calcula la suma y la media
    categorical = categorical.groupby(parent_var).agg(['sum', 'count', 'mean'])
    
    column_names = []
    
    # Iterar a través de las columnas en el nivel 0
    for var in categorical.columns.levels[0]:
        # Iterate through the stats in level 1
        for stat in ['sum', 'count', 'mean']:
            # Iterar a través de las estadísticas en el nivel 1
            column_names.append('%s_%s_%s' % (df_name, var, stat))
    
    categorical.columns = column_names
        
    # Elimina columnas duplicadas por valores
    _, idx = np.unique(categorical, axis = 1, return_index = True)
    categorical = categorical.iloc[:, idx]
    
    return categorical

In [24]:
def aggregate_client(df, group_vars, df_names):
    """Agregar un marco de datos con datos a nivel de préstamo
    a nivel del cliente
    
    Args:
        df (dataframe): datos a nivel de préstamo
        group_vars (lista de dos cadenas): agrupando variables para el préstamo
        y luego el cliente (ejemplo ['SK_ID_PREV', 'SK_ID_CURR'])
        nombres (lista de dos cadenas): nombres para llamar a las columnas resultantes
        (ejemplo ['efectivo', 'cliente'])
        
    Returns:
        df_client (dataframe): estadísticas numéricas agregadas a nivel del cliente.
       Cada cliente tendrá una sola fila con todos los datos numéricos agregados
    """
    
    # Agrega las columnas numéricas
    df_agg = agg_numeric(df, parent_var = group_vars[0], df_name = df_names[0])
    
    # Si hay variables categóricas
    if any(df.dtypes == 'category'):
    
        # Cuenta las columnas categóricas
        df_counts = agg_categorical(df, parent_var = group_vars[0], df_name = df_names[0])

        # Fusiona la numérica y categórica
        df_by_loan = df_counts.merge(df_agg, on = group_vars[0], how = 'outer')

        gc.enable()
        del df_agg, df_counts
        gc.collect()

        # Combina para obtener la identificación del cliente en el marco de datos
        df_by_loan = df_by_loan.merge(df[[group_vars[0], group_vars[1]]], on = group_vars[0], how = 'left')

        # Elimina la identificación del préstamo
        df_by_loan = df_by_loan.drop(columns = [group_vars[0]])

        # Agrega estadísticas numéricas por columna
        df_by_client = agg_numeric(df_by_loan, parent_var = group_vars[1], df_name = df_names[1])

        
    # No hay variables categóricas
    else:
        # Combina para obtener la identificación del cliente en el marco de datos
        df_by_loan = df_agg.merge(df[[group_vars[0], group_vars[1]]], on = group_vars[0], how = 'left')
        
        gc.enable()
        del df_agg
        gc.collect()
        
        # Elimina la identificación del préstamo
        df_by_loan = df_by_loan.drop(columns = [group_vars[0]])
                
        # Agrega estadísticas numéricas por columna
        df_by_client = agg_numeric(df_by_loan, parent_var = group_vars[1], df_name = df_names[1])
        
    # Gestión de la memoria
    gc.enable()
    del df, df_by_loan
    gc.collect()

    return df_by_client

In [21]:
#3-Cargo fichero installments_payments.csv y veo cabecera
installments = pd.read_csv('/Users/LENOVO/Downloads/installments_payments.csv', delimiter= ',' , header=0)
#installments.head()
installmentSNA = installments.dropna()
installmentSNA

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585
...,...,...,...,...,...,...,...,...
13605344,2006721,442291,1.0,3,-1311.0,-1318.0,2934.225,2934.225
13605345,1126000,428449,0.0,12,-301.0,-302.0,6793.470,6750.000
13605346,1519070,444122,1.0,5,-399.0,-407.0,4363.830,4363.830
13605347,2784672,444977,0.0,4,-157.0,-157.0,373.005,373.005


In [22]:
installmentSNA.dtypes.value_counts()

float64    5
int64      3
dtype: int64

In [25]:
##***** Agrega las estadísticas de prestamos por 'SK_ID_CURR'. Elimina columna 'SK_ID_PREV'****
installments_by_client = aggregate_client(installmentSNA, group_vars = ['SK_ID_PREV', 'SK_ID_CURR'], df_names = ['installments', 'client'])
#installments_by_client.head()
installments_by_client

,client_installments_DAYS_ENTRY_PAYMENT_sum_sum,client_installments_DAYS_INSTALMENT_sum_sum,client_installments_DAYS_ENTRY_PAYMENT_min_sum,client_installments_DAYS_INSTALMENT_min_sum,client_installments_DAYS_ENTRY_PAYMENT_mean_sum,client_installments_DAYS_INSTALMENT_mean_sum,client_installments_DAYS_ENTRY_PAYMENT_max_sum,client_installments_DAYS_INSTALMENT_max_sum,client_installments_DAYS_INSTALMENT_sum_min,client_installments_DAYS_ENTRY_PAYMENT_sum_min,...,client_installments_AMT_PAYMENT_min_sum,client_installments_AMT_INSTALMENT_min_sum,client_installments_AMT_PAYMENT_sum_max,client_installments_AMT_INSTALMENT_sum_max,client_installments_AMT_PAYMENT_mean_sum,client_installments_AMT_INSTALMENT_mean_sum,client_installments_AMT_INSTALMENT_max_sum,client_installments_AMT_PAYMENT_max_sum,client_installments_AMT_PAYMENT_sum_sum,client_installments_AMT_INSTALMENT_sum_sum
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,-52813.0,-52598.0,-15608.0,-15584.0,-15365.0,-15314.0,-15080.0,-15044.0,-8658.0,-8647.0,...,27746.775,27746.775,29250.900,29250.900,41195.925,41195.925,8.153775e+04,8.153775e+04,1.528387e+05,1.528387e+05
100002,-113867.0,-106495.0,-11153.0,-10735.0,-5993.0,-5605.0,-931.0,-475.0,-5605.0,-5993.0,...,175783.725,175783.725,219625.695,219625.695,219625.695,219625.695,1.008781e+06,1.008781e+06,4.172888e+06,4.172888e+06
100003,-367137.0,-365546.0,-37757.0,-37514.0,-34633.0,-34454.0,-31594.0,-31394.0,-25740.0,-25821.0,...,1154108.295,1154108.295,1150977.330,1150977.330,1618864.650,1618864.650,4.394101e+06,4.394101e+06,1.134881e+07,1.134881e+07
100004,-6855.0,-6786.0,-2385.0,-2352.0,-2285.0,-2262.0,-2181.0,-2172.0,-2262.0,-2285.0,...,16071.750,16071.750,21288.465,21288.465,21288.465,21288.465,3.172190e+04,3.172190e+04,6.386540e+04,6.386540e+04
100005,-49374.0,-47466.0,-6624.0,-6354.0,-5486.0,-5274.0,-4230.0,-4194.0,-5274.0,-5486.0,...,43318.800,43318.800,56161.845,56161.845,56161.845,56161.845,1.589062e+05,1.589062e+05,5.054566e+05,5.054566e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456251,-7658.0,-5880.0,-1659.0,-1470.0,-1094.0,-840.0,-266.0,-210.0,-840.0,-1094.0,...,46241.370,46241.370,52450.470,52450.470,52450.470,52450.470,8.970507e+04,8.970507e+04,3.671533e+05,3.671533e+05
456252,-86178.0,-86076.0,-14820.0,-14796.0,-14363.0,-14346.0,-13962.0,-13896.0,-14346.0,-14363.0,...,60281.280,60281.280,60419.205,60419.205,60419.205,60419.205,6.044679e+04,6.044679e+04,3.625152e+05,3.625152e+05
456253,-160696.0,-159541.0,-34151.0,-34001.0,-33424.0,-33221.0,-32708.0,-32471.0,-13820.0,-13883.0,...,41805.990,61529.040,33413.760,33413.760,57622.815,61595.910,6.166174e+04,6.166174e+04,3.048921e+05,3.247575e+05


In [26]:
#4-Cargo fichero POS_CASH_balance.csv y veo cabecera
cash = pd.read_csv('/Users/LENOVO/Downloads/POS_CASH_balance.csv', delimiter= ',' , header=0)
#cash.head()
#cash
cashSNA = cash.dropna()
cashSNA

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0
...,...,...,...,...,...,...,...,...
10001353,2448283,226558,-20,6.0,0.0,Active,843,0
10001354,1717234,141565,-19,12.0,0.0,Active,602,0
10001355,1283126,315695,-21,10.0,0.0,Active,609,0
10001356,1082516,450255,-22,12.0,0.0,Active,614,0


In [27]:
cashSNA.dtypes.value_counts()

int64      5
float64    2
object     1
dtype: int64

In [28]:
##***** Agrega las estadísticas de cash por 'SK_ID_CURR'. Elimina columna 'SK_ID_PREV'****
cash_by_client = aggregate_client(cashSNA, group_vars = ['SK_ID_PREV', 'SK_ID_CURR'], df_names = ['cash', 'client'])
#cash_by_client.head()
cash_by_client

,client_cash_MONTHS_BALANCE_sum_sum,client_cash_MONTHS_BALANCE_min_sum,client_cash_MONTHS_BALANCE_mean_sum,client_cash_MONTHS_BALANCE_max_sum,client_cash_MONTHS_BALANCE_sum_min,client_cash_MONTHS_BALANCE_sum_mean,client_cash_MONTHS_BALANCE_sum_max,client_cash_MONTHS_BALANCE_min_min,client_cash_MONTHS_BALANCE_mean_min,client_cash_MONTHS_BALANCE_max_min,...,client_cash_SK_DPD_DEF_sum_sum,client_cash_SK_DPD_max_sum,client_cash_SK_DPD_sum_sum,client_cash_CNT_INSTALMENT_FUTURE_max_sum,client_cash_CNT_INSTALMENT_min_sum,client_cash_CNT_INSTALMENT_mean_sum,client_cash_CNT_INSTALMENT_max_sum,client_cash_MONTHS_BALANCE_count_sum,client_cash_CNT_INSTALMENT_FUTURE_sum_sum,client_cash_CNT_INSTALMENT_sum_sum
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,-2887,-669,-653.0,-637,-378,-320.777778,-275,-96,-94.5,-93,...,28,28,28,28.0,36.0,36.0,36.0,41,62.0,164.0
100002,-3610,-361,-190.0,-19,-190,-190.000000,-190,-19,-10.0,-1,...,0,0,0,456.0,456.0,456.0,456.0,361,5415.0,8664.0
100003,-13240,-1348,-1226.0,-1104,-858,-472.857143,-172,-77,-71.5,-66,...,0,0,0,288.0,248.0,283.0,288.0,272,1608.0,2840.0
100004,-408,-108,-102.0,-96,-102,-102.000000,-102,-27,-25.5,-24,...,0,0,0,16.0,12.0,15.0,16.0,16,36.0,60.0
100005,-1950,-240,-195.0,-150,-195,-195.000000,-195,-24,-19.5,-15,...,0,0,0,120.0,90.0,117.0,120.0,100,720.0,1170.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456251,-288,-64,-36.0,-8,-36,-36.000000,-36,-8,-4.5,-1,...,0,0,0,64.0,56.0,63.0,64.0,64,280.0,504.0
456252,-3871,-574,-553.0,-532,-553,-553.000000,-553,-82,-79.0,-76,...,0,0,0,42.0,42.0,42.0,42.0,49,147.0,294.0
456253,-7746,-1389,-1347.0,-1305,-549,-455.647059,-378,-96,-94.5,-93,...,30,30,30,74.0,114.0,114.0,114.0,101,219.0,630.0


In [29]:
#5-Cargo fichero credit_card_balance.csv y veo cabecera
credit = pd.read_csv('/Users/LENOVO/Downloads/credit_card_balance.csv', delimiter= ',' , header=0)
#credit.head()
#credit
creditSNA = credit.dropna()
creditSNA

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,...,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,...,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,...,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,...,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,...,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3840304,1301378,303373,-15,0.000,0,0.0,0.0,0.0,0.0,0.000,...,0.000,0.000,0.0,0,0.0,0.0,60.0,Active,0,0
3840305,2621524,172581,-11,0.000,0,0.0,0.0,0.0,0.0,0.000,...,0.000,0.000,0.0,0,0.0,0.0,21.0,Active,0,0
3840308,1714892,347207,-9,0.000,45000,0.0,0.0,0.0,0.0,0.000,...,0.000,0.000,0.0,0,0.0,0.0,23.0,Active,0,0
3840309,1302323,215757,-9,275784.975,585000,270000.0,270000.0,0.0,0.0,2250.000,...,273093.975,273093.975,2.0,2,0.0,0.0,18.0,Active,0,0


In [30]:
creditSNA.dtypes.value_counts()

float64    15
int64       7
object      1
dtype: int64

In [31]:
##***** Agrega las estadísticas de credit_card_balance por 'SK_ID_CURR'.Elimina columna 'SK_ID_PREV'
credit_by_client = aggregate_client(creditSNA, group_vars = ['SK_ID_PREV', 'SK_ID_CURR'], df_names = ['credit', 'client'])
#credit_by_client.head()
credit_by_client

,client_credit_MONTHS_BALANCE_sum_sum,client_credit_AMT_RECIVABLE_min_sum,client_credit_AMT_TOTAL_RECEIVABLE_min_sum,client_credit_MONTHS_BALANCE_min_sum,client_credit_MONTHS_BALANCE_mean_sum,client_credit_MONTHS_BALANCE_sum_min,client_credit_MONTHS_BALANCE_sum_mean,client_credit_MONTHS_BALANCE_sum_max,client_credit_AMT_RECIVABLE_min_mean,client_credit_AMT_TOTAL_RECEIVABLE_min_mean,...,client_credit_AMT_TOTAL_RECEIVABLE_max_sum,client_credit_AMT_BALANCE_max_sum,client_credit_AMT_INST_MIN_REGULARITY_sum_sum,client_credit_AMT_PAYMENT_TOTAL_CURRENT_sum_sum,client_credit_AMT_PAYMENT_CURRENT_sum_sum,client_credit_AMT_RECEIVABLE_PRINCIPAL_sum_sum,client_credit_AMT_RECIVABLE_sum_sum,client_credit_AMT_TOTAL_RECEIVABLE_sum_sum,client_credit_AMT_BALANCE_sum_sum,client_credit_AMT_CREDIT_LIMIT_ACTUAL_sum_sum
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100011,-202502,-41124.915,-41124.915,-5402,-2774.0,-2774,-2774.0,-2774,-563.355,-563.355,...,1.347353e+07,1.347353e+07,2.108271e+07,2.441740e+07,2.616223e+07,2.699361e+08,2.802510e+08,2.802510e+08,2.805154e+08,873810000
100013,-356445,-24414.480,-24414.480,-7921,-4005.0,-4005,-4005.0,-4005,-274.320,-274.320,...,1.374793e+07,1.374793e+07,1.152141e+07,5.824592e+07,6.124635e+07,1.334140e+08,1.402892e+08,1.402892e+08,1.407920e+08,1027282500
100028,-37926,0.000,0.000,-1764,-903.0,-903,-903.0,-903,0.000,0.000,...,1.553177e+06,1.568108e+06,1.081925e+07,1.144296e+07,1.182193e+07,1.514466e+07,1.567293e+07,1.567293e+07,1.587498e+07,396900000
100042,-289338,-2726.550,-2726.550,-6889,-3486.0,-3486,-3486.0,-3486,-32.850,-32.850,...,7.728832e+06,7.728832e+06,1.422145e+07,5.138240e+07,5.196025e+07,2.166972e+08,2.261860e+08,2.261860e+08,2.265907e+08,541575000
100043,-15376,0.000,0.000,-961,-496.0,-496,-496.0,-496,0.000,0.000,...,1.342829e+07,1.351171e+07,1.083923e+07,1.371126e+07,2.075466e+07,1.827184e+08,1.875775e+08,1.875775e+08,1.877567e+08,267142500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456242,-196,318571.155,318571.155,-49,-28.0,-28,-28.0,-28,45510.165,45510.165,...,1.518310e+06,1.520924e+06,3.908885e+05,1.096489e+06,1.096489e+06,8.004955e+06,8.178509e+06,8.178509e+06,8.198858e+06,11025000
456244,-32800,0.000,0.000,-1600,-820.0,-820,-820.0,-820,0.000,0.000,...,1.814511e+07,1.814511e+07,1.042272e+07,5.366169e+07,5.366169e+07,1.969596e+08,2.020655e+08,2.020655e+08,2.036550e+08,468000000
456246,-245,0.000,0.000,-56,-35.0,-35,-35.0,-35,0.000,0.000,...,3.044308e+05,3.044308e+05,8.059243e+04,8.710430e+05,9.201355e+05,5.387206e+05,5.395537e+05,5.395537e+05,5.472602e+05,6615000


In [33]:
#6-Cargo fichero previous_application.csv y veo cabecera
previous = pd.read_csv('/Users/LENOVO/Downloads/previous_application.csv', delimiter= ',' , header=0)
#previous.head()
previous
#previousSNA = previous.dropna()
#previousSNA

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1670209,2300464,352015,Consumer loans,14704.290,267295.5,311400.0,0.0,267295.5,WEDNESDAY,12,...,Furniture,30.0,low_normal,POS industry with interest,365243.0,-508.0,362.0,-358.0,-351.0,0.0
1670210,2357031,334635,Consumer loans,6622.020,87750.0,64291.5,29250.0,87750.0,TUESDAY,15,...,Furniture,12.0,middle,POS industry with interest,365243.0,-1604.0,-1274.0,-1304.0,-1297.0,0.0
1670211,2659632,249544,Consumer loans,11520.855,105237.0,102523.5,10525.5,105237.0,MONDAY,12,...,Consumer electronics,10.0,low_normal,POS household with interest,365243.0,-1457.0,-1187.0,-1187.0,-1181.0,0.0
1670212,2785582,400317,Cash loans,18821.520,180000.0,191880.0,NaN,180000.0,WEDNESDAY,9,...,XNA,12.0,low_normal,Cash X-Sell: low,365243.0,-1155.0,-825.0,-825.0,-817.0,1.0


In [34]:
previous.dtypes.value_counts()
#previousSNA.dtypes.value_counts()

object     16
float64    15
int64       6
dtype: int64

In [35]:
# Calculate aggregate statistics for each numeric column
previous_agg = agg_numeric(previous, 'SK_ID_CURR', 'previous')
#previous_agg.head()
previous_agg

,previous_DAYS_DECISION_sum,previous_DAYS_DECISION_min,previous_DAYS_DECISION_mean,previous_DAYS_DECISION_max,previous_DAYS_FIRST_DUE_sum,previous_DAYS_FIRST_DUE_min,previous_DAYS_FIRST_DUE_mean,previous_DAYS_FIRST_DUE_max,previous_DAYS_LAST_DUE_sum,previous_DAYS_LAST_DUE_min,...,previous_DAYS_FIRST_DRAWING_min,previous_DAYS_FIRST_DRAWING_mean,previous_DAYS_FIRST_DRAWING_max,previous_DAYS_FIRST_DRAWING_sum,previous_RATE_INTEREST_PRIMARY_min,previous_RATE_INTEREST_PRIMARY_mean,previous_RATE_INTEREST_PRIMARY_max,previous_RATE_INTEREST_PRIVILEGED_min,previous_RATE_INTEREST_PRIVILEGED_mean,previous_RATE_INTEREST_PRIVILEGED_max
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,-1740,-1740,-1740.000,-1740,-1709.0,-1709.0,-1709.000000,-1709.0,-1619.0,-1619.0,...,365243.0,365243.0,365243.0,365243.0,NaN,NaN,NaN,NaN,NaN,NaN
100002,-606,-606,-606.000,-606,-565.0,-565.0,-565.000000,-565.0,-25.0,-25.0,...,365243.0,365243.0,365243.0,365243.0,NaN,NaN,NaN,NaN,NaN,NaN
100003,-3915,-2341,-1305.000,-746,-3823.0,-2310.0,-1274.333333,-716.0,-3163.0,-1980.0,...,365243.0,365243.0,365243.0,1095729.0,NaN,NaN,NaN,NaN,NaN,NaN
100004,-815,-815,-815.000,-815,-784.0,-784.0,-784.000000,-784.0,-724.0,-724.0,...,365243.0,365243.0,365243.0,365243.0,NaN,NaN,NaN,NaN,NaN,NaN
100005,-1072,-757,-536.000,-315,-706.0,-706.0,-706.000000,-706.0,-466.0,-466.0,...,365243.0,365243.0,365243.0,365243.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456251,-273,-273,-273.000,-273,-210.0,-210.0,-210.000000,-210.0,-30.0,-30.0,...,365243.0,365243.0,365243.0,365243.0,NaN,NaN,NaN,NaN,NaN,NaN
456252,-2497,-2497,-2497.000,-2497,-2466.0,-2466.0,-2466.000000,-2466.0,-2316.0,-2316.0,...,365243.0,365243.0,365243.0,365243.0,NaN,NaN,NaN,NaN,NaN,NaN
456253,-4760,-2851,-2380.000,-1909,-4678.0,-2812.0,-2339.000000,-1866.0,-4438.0,-2722.0,...,365243.0,365243.0,365243.0,730486.0,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
# Calculate value counts for each categorical column
previous_counts = count_categoricas(previous, 'SK_ID_CURR', 'previous')
#previous_counts.head()
previous_counts

,previous_NAME_CONTRACT_TYPE_Cash loans_count,previous_NAME_CONTRACT_TYPE_Cash loans_count_norm,previous_NAME_CONTRACT_TYPE_Consumer loans_count,previous_NAME_CONTRACT_TYPE_Consumer loans_count_norm,previous_NAME_CONTRACT_TYPE_Revolving loans_count,previous_NAME_CONTRACT_TYPE_Revolving loans_count_norm,previous_NAME_CONTRACT_TYPE_XNA_count,previous_NAME_CONTRACT_TYPE_XNA_count_norm,previous_WEEKDAY_APPR_PROCESS_START_FRIDAY_count,previous_WEEKDAY_APPR_PROCESS_START_FRIDAY_count_norm,...,previous_PRODUCT_COMBINATION_POS industry without interest_count,previous_PRODUCT_COMBINATION_POS industry without interest_count_norm,previous_PRODUCT_COMBINATION_POS mobile with interest_count,previous_PRODUCT_COMBINATION_POS mobile with interest_count_norm,previous_PRODUCT_COMBINATION_POS mobile without interest_count,previous_PRODUCT_COMBINATION_POS mobile without interest_count_norm,previous_PRODUCT_COMBINATION_POS other with interest_count,previous_PRODUCT_COMBINATION_POS other with interest_count_norm,previous_PRODUCT_COMBINATION_POS others without interest_count,previous_PRODUCT_COMBINATION_POS others without interest_count_norm
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,0,0.000000,1,1.000000,0,0.000,0,0.0,1,1.000000,...,0,0.0,1,1.00,0,0.0,0,0.0,0,0.0
100002,0,0.000000,1,1.000000,0,0.000,0,0.0,0,0.000000,...,0,0.0,0,0.00,0,0.0,1,1.0,0,0.0
100003,1,0.333333,2,0.666667,0,0.000,0,0.0,1,0.333333,...,0,0.0,0,0.00,0,0.0,0,0.0,0,0.0
100004,0,0.000000,1,1.000000,0,0.000,0,0.0,1,1.000000,...,0,0.0,0,0.00,1,1.0,0,0.0,0,0.0
100005,1,0.500000,1,0.500000,0,0.000,0,0.0,1,0.500000,...,0,0.0,1,0.50,0,0.0,0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456251,0,0.000000,1,1.000000,0,0.000,0,0.0,0,0.000000,...,0,0.0,1,1.00,0,0.0,0,0.0,0,0.0
456252,0,0.000000,1,1.000000,0,0.000,0,0.0,0,0.000000,...,0,0.0,0,0.00,0,0.0,0,0.0,0,0.0
456253,0,0.000000,2,1.000000,0,0.000,0,0.0,0,0.000000,...,0,0.0,2,1.00,0,0.0,0,0.0,0,0.0


In [37]:
# Dataframe agrupando BureauBalance_agg con BureauBalance_counts por préstamo anterior
previous_application_agg = previous_agg.merge(previous_counts, right_index = True, left_on = 'SK_ID_CURR', how = 'outer')
previous_application_agg

,previous_DAYS_DECISION_sum,previous_DAYS_DECISION_min,previous_DAYS_DECISION_mean,previous_DAYS_DECISION_max,previous_DAYS_FIRST_DUE_sum,previous_DAYS_FIRST_DUE_min,previous_DAYS_FIRST_DUE_mean,previous_DAYS_FIRST_DUE_max,previous_DAYS_LAST_DUE_sum,previous_DAYS_LAST_DUE_min,...,previous_PRODUCT_COMBINATION_POS industry without interest_count,previous_PRODUCT_COMBINATION_POS industry without interest_count_norm,previous_PRODUCT_COMBINATION_POS mobile with interest_count,previous_PRODUCT_COMBINATION_POS mobile with interest_count_norm,previous_PRODUCT_COMBINATION_POS mobile without interest_count,previous_PRODUCT_COMBINATION_POS mobile without interest_count_norm,previous_PRODUCT_COMBINATION_POS other with interest_count,previous_PRODUCT_COMBINATION_POS other with interest_count_norm,previous_PRODUCT_COMBINATION_POS others without interest_count,previous_PRODUCT_COMBINATION_POS others without interest_count_norm
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,-1740,-1740,-1740.000,-1740,-1709.0,-1709.0,-1709.000000,-1709.0,-1619.0,-1619.0,...,0,0.0,1,1.00,0,0.0,0,0.0,0,0.0
100002,-606,-606,-606.000,-606,-565.0,-565.0,-565.000000,-565.0,-25.0,-25.0,...,0,0.0,0,0.00,0,0.0,1,1.0,0,0.0
100003,-3915,-2341,-1305.000,-746,-3823.0,-2310.0,-1274.333333,-716.0,-3163.0,-1980.0,...,0,0.0,0,0.00,0,0.0,0,0.0,0,0.0
100004,-815,-815,-815.000,-815,-784.0,-784.0,-784.000000,-784.0,-724.0,-724.0,...,0,0.0,0,0.00,1,1.0,0,0.0,0,0.0
100005,-1072,-757,-536.000,-315,-706.0,-706.0,-706.000000,-706.0,-466.0,-466.0,...,0,0.0,1,0.50,0,0.0,0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456251,-273,-273,-273.000,-273,-210.0,-210.0,-210.000000,-210.0,-30.0,-30.0,...,0,0.0,1,1.00,0,0.0,0,0.0,0,0.0
456252,-2497,-2497,-2497.000,-2497,-2466.0,-2466.0,-2466.000000,-2466.0,-2316.0,-2316.0,...,0,0.0,0,0.00,0,0.0,0,0.0,0,0.0
456253,-4760,-2851,-2380.000,-1909,-4678.0,-2812.0,-2339.000000,-1866.0,-4438.0,-2722.0,...,0,0.0,2,1.00,0,0.0,0,0.0,0,0.0


In [38]:
#7-Cargo fichero application_train.csv y veo cabecera
Train = pd.read_csv('/Users/LENOVO/Downloads/application_train.csv', delimiter= ',' , header=0)
Train

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
Train.dtypes.value_counts()

float64    65
int64      41
object     16
dtype: int64

In [41]:
#8-Cargo fichero application_test.csv y veo cabecera
Test = pd.read_csv('/Users/LENOVO/Downloads/application_test.csv', delimiter= ',' , header=0)
Test

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,456221,Cash loans,F,N,Y,0,121500.0,412560.0,17473.5,270000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
48740,456222,Cash loans,F,N,N,2,157500.0,622413.0,31909.5,495000.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
48741,456223,Cash loans,F,Y,Y,1,202500.0,315000.0,33205.5,315000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,3.0,1.0
48742,456224,Cash loans,M,N,N,0,225000.0,450000.0,25128.0,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0


In [42]:
Test.dtypes.value_counts()

float64    65
int64      40
object     16
dtype: int64

In [43]:
# Create a label encoder object
le = preprocessing.LabelEncoder()
le_count = 0

In [44]:
# Iterate through the columns
for col in Train:
    if Train[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(Train[col].unique())) <= 2:
            # Train on the training data
            le.fit(Train[col])
            # Transform both training and testing data
            Train[col] = le.transform(Train[col])
            Test[col] = le.transform(Test[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
            
print('%d columns were label encoded.' % le_count)

3 columns were label encoded.


In [45]:
# one-hot encoding of categorical variables
Train = pd.get_dummies(Train)
Test  = pd.get_dummies(Test )

print('Training Features shape: ', Train.shape)
print('Testing Features shape: ', Test.shape)

Training Features shape:  (307511, 243)
Testing Features shape:  (48744, 239)


In [46]:
Train_labels = Train['TARGET']

# Align the training and testing data, keep only columns present in both dataframes
Train, Test = Train.align(Test, join = 'inner', axis = 1)

# Add the target back in
Train['TARGET'] = Train_labels

print('Training Features shape: ', Train.shape)
print('Testing Features shape: ', Test.shape)

Training Features shape:  (307511, 240)
Testing Features shape:  (48744, 239)


In [47]:
# Join to the training dataframe BureauBureauBalance_by_client
#train = pd.read_csv('../input/application_train.csv')
Train = Train.merge(BureauBureauBalance_by_client, on = 'SK_ID_CURR', how = 'left')

In [48]:
# Fill the missing values with 0 
Train['SK_ID_CURR'] = Train['SK_ID_CURR'].fillna(0)#
#Train.head()
Train

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,client_bureau_balance_STATUS_X_count_count,client_bureau_balance_STATUS_X_count_mean,client_bureau_balance_STATUS_X_count_max,client_bureau_balance_STATUS_X_count_min,client_bureau_balance_STATUS_X_count_sum,client_bureau_balance_STATUS_X_count_norm_count,client_bureau_balance_STATUS_X_count_norm_mean,client_bureau_balance_STATUS_X_count_norm_max,client_bureau_balance_STATUS_X_count_norm_min,client_bureau_balance_STATUS_X_count_norm_sum
0,100002,0,0,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,...,2.0,1.5,3.0,0.0,3.0,2.0,0.09375,0.1875,0.0,0.1875
1,100003,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004,1,1,1,0,67500.0,135000.0,6750.0,135000.0,0.010032,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,0,0,1,0,135000.0,312682.5,29686.5,297000.0,0.008019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,0,1,0,121500.0,513000.0,21865.5,513000.0,0.028663,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,0,0,0,157500.0,254700.0,27558.0,225000.0,0.032561,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,0,0,1,0,72000.0,269550.0,12001.5,225000.0,0.025164,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,0,0,1,0,153000.0,677664.0,29979.0,585000.0,0.005002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307509,456254,0,0,1,0,171000.0,370107.0,20205.0,319500.0,0.005313,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
Train.dtypes.value_counts()

float64    170
uint8      131
int64       41
int32        3
dtype: int64

In [106]:
#Train.select_dtypes('float64').apply(pd.Series.nunique, axis = 0)

In [50]:
correlations = Train.corr()['TARGET'].sort_values()
correlations

EXT_SOURCE_3                                    -0.178919
EXT_SOURCE_2                                    -0.160472
EXT_SOURCE_1                                    -0.155317
NAME_EDUCATION_TYPE_Higher education            -0.056593
CODE_GENDER_F                                   -0.054704
                                                   ...   
REGION_RATING_CLIENT                             0.058899
client_bureau_balance_STATUS_1_count_norm_max    0.058953
REGION_RATING_CLIENT_W_CITY                      0.060893
DAYS_BIRTH                                       0.078239
TARGET                                           1.000000
Name: TARGET, Length: 345, dtype: float64

In [51]:
# Join to the training dataframe Bureau_agg
#train = pd.read_csv('../input/application_train.csv')
Train = Train.merge(Bureau_agg, on = 'SK_ID_CURR', how = 'left')
Train

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,"(DAYS_CREDIT_UPDATE, count)","(DAYS_CREDIT_UPDATE, mean)","(DAYS_CREDIT_UPDATE, max)","(DAYS_CREDIT_UPDATE, min)","(DAYS_CREDIT_UPDATE, sum)","(AMT_ANNUITY, count)","(AMT_ANNUITY, mean)","(AMT_ANNUITY, max)","(AMT_ANNUITY, min)","(AMT_ANNUITY, sum)"
0,100002,0,0,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,...,2.0,-609.5,-34.0,-1185.0,-1219.0,2.0,0.0,0.0,0.0,0.0
1,100003,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004,1,1,1,0,67500.0,135000.0,6750.0,135000.0,0.010032,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,0,0,1,0,135000.0,312682.5,29686.5,297000.0,0.008019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,0,1,0,121500.0,513000.0,21865.5,513000.0,0.028663,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,0,0,0,157500.0,254700.0,27558.0,225000.0,0.032561,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,0,0,1,0,72000.0,269550.0,12001.5,225000.0,0.025164,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,0,0,1,0,153000.0,677664.0,29979.0,585000.0,0.005002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307509,456254,0,0,1,0,171000.0,370107.0,20205.0,319500.0,0.005313,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
Train.dtypes.value_counts()

float64    230
uint8      131
int64       41
int32        3
dtype: int64

In [110]:
#Train.select_dtypes('float64').apply(pd.Series.nunique, axis = 0)

In [53]:
correlations = Train.corr()['TARGET'].sort_values()
correlations

EXT_SOURCE_3                           -0.178919
EXT_SOURCE_2                           -0.160472
EXT_SOURCE_1                           -0.155317
NAME_EDUCATION_TYPE_Higher education   -0.056593
CODE_GENDER_F                          -0.054704
                                          ...   
TARGET                                  1.000000
(AMT_CREDIT_SUM_OVERDUE, mean)               NaN
(AMT_CREDIT_SUM_OVERDUE, max)                NaN
(AMT_CREDIT_SUM_OVERDUE, min)                NaN
(AMT_CREDIT_SUM_OVERDUE, sum)                NaN
Name: TARGET, Length: 405, dtype: float64

In [54]:
# Join to the training dataframe installments_by_client
#train = pd.read_csv('../input/application_train.csv')
Train = Train.merge(installments_by_client, on = 'SK_ID_CURR', how = 'left')
Train

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,client_installments_AMT_PAYMENT_min_sum,client_installments_AMT_INSTALMENT_min_sum,client_installments_AMT_PAYMENT_sum_max,client_installments_AMT_INSTALMENT_sum_max,client_installments_AMT_PAYMENT_mean_sum,client_installments_AMT_INSTALMENT_mean_sum,client_installments_AMT_INSTALMENT_max_sum,client_installments_AMT_PAYMENT_max_sum,client_installments_AMT_PAYMENT_sum_sum,client_installments_AMT_INSTALMENT_sum_sum
0,100002,0,0,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,...,175783.725,175783.725,219625.695,219625.695,219625.695,219625.695,1.008781e+06,1.008781e+06,4.172888e+06,4.172888e+06
1,100003,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,...,1154108.295,1154108.295,1150977.330,1150977.330,1618864.650,1618864.650,4.394101e+06,4.394101e+06,1.134881e+07,1.134881e+07
2,100004,1,1,1,0,67500.0,135000.0,6750.0,135000.0,0.010032,...,16071.750,16071.750,21288.465,21288.465,21288.465,21288.465,3.172190e+04,3.172190e+04,6.386540e+04,6.386540e+04
3,100006,0,0,1,0,135000.0,312682.5,29686.5,297000.0,0.008019,...,994476.690,994476.690,691786.890,691786.890,1007153.415,1007153.415,1.057860e+06,1.057860e+06,3.719996e+06,3.719996e+06
4,100007,0,0,1,0,121500.0,513000.0,21865.5,513000.0,0.028663,...,483756.390,825845.850,280199.700,294631.110,806127.975,835985.340,8.367034e+05,8.367034e+05,1.139621e+07,1.179154e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,0,0,0,157500.0,254700.0,27558.0,225000.0,0.032561,...,46241.370,46241.370,52450.470,52450.470,52450.470,52450.470,8.970507e+04,8.970507e+04,3.671533e+05,3.671533e+05
307507,456252,0,0,1,0,72000.0,269550.0,12001.5,225000.0,0.025164,...,60281.280,60281.280,60419.205,60419.205,60419.205,60419.205,6.044679e+04,6.044679e+04,3.625152e+05,3.625152e+05
307508,456253,0,0,1,0,153000.0,677664.0,29979.0,585000.0,0.005002,...,41805.990,61529.040,33413.760,33413.760,57622.815,61595.910,6.166174e+04,6.166174e+04,3.048921e+05,3.247575e+05
307509,456254,0,0,1,0,171000.0,370107.0,20205.0,319500.0,0.005313,...,194556.825,194556.825,171592.425,171592.425,194556.825,194556.825,1.945568e+05,1.945568e+05,1.773976e+06,1.773976e+06


In [55]:
Train.dtypes.value_counts()

float64    331
uint8      131
int64       41
int32        3
dtype: int64

In [114]:
#Train.select_dtypes('float64').apply(pd.Series.nunique, axis = 0)

In [56]:
correlations = Train.corr()['TARGET'].sort_values()
correlations

EXT_SOURCE_3                              -0.178919
EXT_SOURCE_2                              -0.160472
EXT_SOURCE_1                              -0.155317
client_installments_AMT_PAYMENT_min_sum   -0.058453
NAME_EDUCATION_TYPE_Higher education      -0.056593
                                             ...   
TARGET                                     1.000000
(AMT_CREDIT_SUM_OVERDUE, mean)                  NaN
(AMT_CREDIT_SUM_OVERDUE, max)                   NaN
(AMT_CREDIT_SUM_OVERDUE, min)                   NaN
(AMT_CREDIT_SUM_OVERDUE, sum)                   NaN
Name: TARGET, Length: 506, dtype: float64

In [57]:
# Join to the training dataframe cash_by_client
#train = pd.read_csv('../input/application_train.csv')
Train = Train.merge(cash_by_client, on = 'SK_ID_CURR', how = 'left')
Train

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,client_cash_SK_DPD_DEF_sum_sum,client_cash_SK_DPD_max_sum,client_cash_SK_DPD_sum_sum,client_cash_CNT_INSTALMENT_FUTURE_max_sum,client_cash_CNT_INSTALMENT_min_sum,client_cash_CNT_INSTALMENT_mean_sum,client_cash_CNT_INSTALMENT_max_sum,client_cash_MONTHS_BALANCE_count_sum,client_cash_CNT_INSTALMENT_FUTURE_sum_sum,client_cash_CNT_INSTALMENT_sum_sum
0,100002,0,0,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,...,0.0,0.0,0.0,456.0,456.0,456.0,456.0,361.0,5415.0,8664.0
1,100003,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,...,0.0,0.0,0.0,288.0,248.0,283.0,288.0,272.0,1608.0,2840.0
2,100004,1,1,1,0,67500.0,135000.0,6750.0,135000.0,0.010032,...,0.0,0.0,0.0,16.0,12.0,15.0,16.0,16.0,36.0,60.0
3,100006,0,0,1,0,135000.0,312682.5,29686.5,297000.0,0.008019,...,0.0,0.0,0.0,348.0,158.0,240.0,348.0,158.0,1244.0,1840.0
4,100007,0,0,1,0,121500.0,513000.0,21865.5,513000.0,0.028663,...,0.0,0.0,0.0,1024.0,1006.0,1012.0,1024.0,906.0,8104.0,14128.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,0,0,0,157500.0,254700.0,27558.0,225000.0,0.032561,...,0.0,0.0,0.0,64.0,56.0,63.0,64.0,64.0,280.0,504.0
307507,456252,0,0,1,0,72000.0,269550.0,12001.5,225000.0,0.025164,...,0.0,0.0,0.0,42.0,42.0,42.0,42.0,49.0,147.0,294.0
307508,456253,0,0,1,0,153000.0,677664.0,29979.0,585000.0,0.005002,...,30.0,30.0,30.0,74.0,114.0,114.0,114.0,101.0,219.0,630.0
307509,456254,0,0,1,0,171000.0,370107.0,20205.0,319500.0,0.005313,...,0.0,0.0,0.0,298.0,298.0,298.0,298.0,202.0,2061.0,2990.0


In [58]:
Train.dtypes.value_counts()

float64    416
uint8      131
int64       41
int32        3
dtype: int64

In [118]:
#Train.select_dtypes('float64').apply(pd.Series.nunique, axis = 0)

In [59]:
correlations = Train.corr()['TARGET'].sort_values()
correlations

EXT_SOURCE_3                              -0.178919
EXT_SOURCE_2                              -0.160472
EXT_SOURCE_1                              -0.155317
client_installments_AMT_PAYMENT_min_sum   -0.058453
NAME_EDUCATION_TYPE_Higher education      -0.056593
                                             ...   
TARGET                                     1.000000
(AMT_CREDIT_SUM_OVERDUE, mean)                  NaN
(AMT_CREDIT_SUM_OVERDUE, max)                   NaN
(AMT_CREDIT_SUM_OVERDUE, min)                   NaN
(AMT_CREDIT_SUM_OVERDUE, sum)                   NaN
Name: TARGET, Length: 591, dtype: float64

In [60]:
# Join to the training dataframe credit_by_client
#train = pd.read_csv('../input/application_train.csv')
Train = Train.merge(credit_by_client, on = 'SK_ID_CURR', how = 'left')
Train

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,client_credit_AMT_TOTAL_RECEIVABLE_max_sum,client_credit_AMT_BALANCE_max_sum,client_credit_AMT_INST_MIN_REGULARITY_sum_sum,client_credit_AMT_PAYMENT_TOTAL_CURRENT_sum_sum,client_credit_AMT_PAYMENT_CURRENT_sum_sum,client_credit_AMT_RECEIVABLE_PRINCIPAL_sum_sum,client_credit_AMT_RECIVABLE_sum_sum,client_credit_AMT_TOTAL_RECEIVABLE_sum_sum,client_credit_AMT_BALANCE_sum_sum,client_credit_AMT_CREDIT_LIMIT_ACTUAL_sum_sum
0,100002,0,0,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004,1,1,1,0,67500.0,135000.0,6750.0,135000.0,0.010032,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,0,0,1,0,135000.0,312682.5,29686.5,297000.0,0.008019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,0,1,0,121500.0,513000.0,21865.5,513000.0,0.028663,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,0,0,0,157500.0,254700.0,27558.0,225000.0,0.032561,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,0,0,1,0,72000.0,269550.0,12001.5,225000.0,0.025164,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,0,0,1,0,153000.0,677664.0,29979.0,585000.0,0.005002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307509,456254,0,0,1,0,171000.0,370107.0,20205.0,319500.0,0.005313,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
Train.dtypes.value_counts()

float64    731
uint8      131
int64       41
int32        3
dtype: int64

In [122]:
#Train.select_dtypes('float64').apply(pd.Series.nunique, axis = 0)

In [62]:
correlations = Train.corr()['TARGET'].sort_values()
correlations

EXT_SOURCE_3                              -0.178919
EXT_SOURCE_2                              -0.160472
EXT_SOURCE_1                              -0.155317
client_credit_MONTHS_BALANCE_count_min    -0.108704
client_credit_MONTHS_BALANCE_count_mean   -0.108084
                                             ...   
(AMT_CREDIT_SUM_OVERDUE, max)                   NaN
(AMT_CREDIT_SUM_OVERDUE, min)                   NaN
(AMT_CREDIT_SUM_OVERDUE, sum)                   NaN
client_credit_SK_DPD_DEF_min_mean               NaN
client_credit_SK_DPD_DEF_min_sum                NaN
Name: TARGET, Length: 906, dtype: float64

In [124]:
#Remove variables to free memory
#gc.enable()
#del previous, previous_agg, previous_counts
#gc.collect()

In [63]:
# Join to the training dataframe previous_application_agg
#train = pd.read_csv('../input/application_train.csv')
Train = Train.merge(previous_application_agg, on = 'SK_ID_CURR', how = 'left')
Train

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,previous_PRODUCT_COMBINATION_POS industry without interest_count,previous_PRODUCT_COMBINATION_POS industry without interest_count_norm,previous_PRODUCT_COMBINATION_POS mobile with interest_count,previous_PRODUCT_COMBINATION_POS mobile with interest_count_norm,previous_PRODUCT_COMBINATION_POS mobile without interest_count,previous_PRODUCT_COMBINATION_POS mobile without interest_count_norm,previous_PRODUCT_COMBINATION_POS other with interest_count,previous_PRODUCT_COMBINATION_POS other with interest_count_norm,previous_PRODUCT_COMBINATION_POS others without interest_count,previous_PRODUCT_COMBINATION_POS others without interest_count_norm
0,100002,0,0,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,...,0.0,0.0,0.0,0.000000,0.0,0.0,1.0,1.0,0.0,0.0
1,100003,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,1,1,1,0,67500.0,135000.0,6750.0,135000.0,0.010032,...,0.0,0.0,0.0,0.000000,1.0,1.0,0.0,0.0,0.0,0.0
3,100006,0,0,1,0,135000.0,312682.5,29686.5,297000.0,0.008019,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,100007,0,0,1,0,121500.0,513000.0,21865.5,513000.0,0.028663,...,0.0,0.0,1.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,0,0,0,157500.0,254700.0,27558.0,225000.0,0.032561,...,0.0,0.0,1.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0
307507,456252,0,0,1,0,72000.0,269550.0,12001.5,225000.0,0.025164,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
307508,456253,0,0,1,0,153000.0,677664.0,29979.0,585000.0,0.005002,...,0.0,0.0,2.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0
307509,456254,0,0,1,0,171000.0,370107.0,20205.0,319500.0,0.005313,...,0.0,0.0,1.0,0.500000,0.0,0.0,0.0,0.0,0.0,0.0


In [64]:
Train.dtypes.value_counts()

float64    1101
uint8       131
int64        41
int32         3
dtype: int64

In [127]:
#Train.select_dtypes('float64').apply(pd.Series.nunique, axis = 0)

In [65]:
correlations = Train.corr()['TARGET'].sort_values()
correlations

EXT_SOURCE_3                                                 -0.178919
EXT_SOURCE_2                                                 -0.160472
EXT_SOURCE_1                                                 -0.155317
client_credit_MONTHS_BALANCE_count_min                       -0.108704
client_credit_MONTHS_BALANCE_count_mean                      -0.108084
                                                                ...   
(AMT_CREDIT_SUM_OVERDUE, sum)                                      NaN
client_credit_SK_DPD_DEF_min_mean                                  NaN
client_credit_SK_DPD_DEF_min_sum                                   NaN
previous_NAME_GOODS_CATEGORY_House Construction_count              NaN
previous_NAME_GOODS_CATEGORY_House Construction_count_norm         NaN
Name: TARGET, Length: 1276, dtype: float64

In [66]:
# Display correlations
print('Most Positive Correlations:\n', correlations.tail(15))
print('\nMost Negative Correlations:\n', correlations.head(15))

Most Positive Correlations:
 client_credit_MONTHS_BALANCE_min_min                          0.108040
client_credit_MONTHS_BALANCE_mean_min                         0.108127
client_credit_MONTHS_BALANCE_min_mean                         0.108188
client_credit_MONTHS_BALANCE_mean_mean                        0.108274
client_credit_MONTHS_BALANCE_min_max                          0.108819
client_credit_MONTHS_BALANCE_mean_max                         0.108903
TARGET                                                        1.000000
(AMT_CREDIT_SUM_OVERDUE, mean)                                     NaN
(AMT_CREDIT_SUM_OVERDUE, max)                                      NaN
(AMT_CREDIT_SUM_OVERDUE, min)                                      NaN
(AMT_CREDIT_SUM_OVERDUE, sum)                                      NaN
client_credit_SK_DPD_DEF_min_mean                                  NaN
client_credit_SK_DPD_DEF_min_sum                                   NaN
previous_NAME_GOODS_CATEGORY_House Construction_